In [ ]:
!pip install lxml
!pip install bs4
!conda install -c conda-forge folium=0.5.0 --yes
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import folium

Solving environment: | 

In [ ]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(source,'lxml')

df = pd.read_html(str(soup.table))
df=df[0]
df.head()

In [ ]:
df = df[df.Borough != 'Not assigned']
df = df.groupby(['Postal Code','Borough'], sort=False).agg(', '.join)
df.reset_index(inplace=True)
df['Neighborhood'] = np.where(df['Neighborhood'] == 'Not assigned',df['Borough'], df['Neighborhood'])

df.head()

In [ ]:
lat_lon = pd.read_csv('https://cocl.us/Geospatial_data')
lat_lon.head()

In [ ]:
lat_lon.rename(columns={'Postcode':'Postal Code'},inplace=True)
df = pd.merge(df,lat_lon,on='Postal Code')
df.head()

From here clustering and visualization is performed

In [ ]:
df = df[df['Borough'].str.contains('Toronto',regex=False)]
df.head()

In [ ]:
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighborhood in zip(df['Latitude'],df['Longitude'],df['Borough'],df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

In [ ]:
from sklearn.cluster import KMeans
k=5
toronto_clustering = df.drop(['Postal Code','Borough','Neighborhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_
df.insert(0, 'Cluster Labels', kmeans.labels_)

In [ ]:
df.head()

In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, neighborhood, cluster in zip(df['Latitude'], df['Longitude'], df['Neighborhood'], df['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters